# scraper Clasificados de los tiempos

notebook para obtenere datos de la página de los clasificados de los tiempos 

LINK: https://clasificados.lostiempos.com/inmuebles 

In [1]:
import pandas as pd
import numpy as np
import requests
import re
import math
from datetime import datetime
from pyquery import PyQuery as pq


In [2]:
BASE_URL = "https://clasificados.lostiempos.com"
URL = f"{BASE_URL}/inmuebles?sort_by=created&sort_order=DESC&page="

HEADERS = {'User-Agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) "
                         "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50"
                         ".0.2661.102 Safari/537.36"}
posts = []

MONTHS_SPANISH = ["Enero", "Febrero", "Marzo", "Abril", "Mayo", "Junio", "Julio", "Agosto", "Septiembre", "Octubre", "Noviembre", "Diciembre"]

response = requests.get(f"{URL}{0}", headers=HEADERS)
doc = pq(response.content)

In [4]:
def append_elements(doc, posts):
    l = doc.find('.views-row')
    for i, element in enumerate(l):
        data = get_data(l.eq(i))
        posts.append(data)

In [5]:
# clean the text
def clean_text(s):
    return s.replace('\n', '').strip()

# get data from text like 'Publicado: 23 Marzo 2023'
def get_date(s):
    original_date = s.replace('Publicado: ', '')
    dt = original_date.split(' ')
    return f"{dt[2]}-{str(MONTHS_SPANISH.index(dt[1])).zfill(2)}-{dt[0].zfill(2)}"

# Get celphone and phone number 
def get_phones(s):
    return re.findall(r"([\d]{8}|[\d]{7})", s)

def get_price(s):
    # TODO: hacer el calculo de precios.
    pass

In [6]:
def get_data(element):
    title = element.children('.title')[0].text_content()
    date = element.children('.publish-date')[0].text_content()
    description = element.children('.body')[0].text_content()
    price = element.children('.ads-price')[0].text_content()
    keys = element.children('.description')[0].text_content()
    path = element.children('.buttons a')[0].attrib['href']

    return {
        'title': clean_text(title),
        'date': get_date(clean_text(date)),
        'description': clean_text(description),
        'price': clean_text(price),
        'keys': clean_text(keys),
        'path': clean_text(path),
    }

# get_data(doc.find('.views-row').eq(0))

In [7]:
doc.find('.views-row').eq(0).children('.buttons a')[0].attrib['href']

'/anuncio/inmuebles/galpon-en-venta-400/2019423'

In [8]:
number_elements = int(re.findall(r'\d+', doc.find('.resume-search')[0].text_content())[0])
NUMBER_OF_PAGES = math.ceil(number_elements / 20)
print("total:", number_elements, "- paginas:", NUMBER_OF_PAGES)

append_elements(doc, posts)

total: 62 - paginas: 4


In [9]:
for n in range(1, NUMBER_OF_PAGES):
    print("get data from:", f"{URL}{n}")
    response = requests.get(f"{URL}{n}", headers=HEADERS)
    doc = pq(response.content)
    append_elements(doc, posts)

print("total de publicaciones:", len(posts))

get data from: https://clasificados.lostiempos.com/inmuebles?sort_by=created&sort_order=DESC&page=1
get data from: https://clasificados.lostiempos.com/inmuebles?sort_by=created&sort_order=DESC&page=2
get data from: https://clasificados.lostiempos.com/inmuebles?sort_by=created&sort_order=DESC&page=3
total de publicaciones: 62


In [10]:
df = pd.DataFrame(posts)
df

,title,date,description,price,keys,path
0,Galpon En Venta 400,2023-06-20,"GALPON En venta, 400 m2, zona Pacata, inmedia...",,Cochabamba - Venta - Depósitos Galpones,/anuncio/inmuebles/galpon-en-venta-400/2019423
1,39 800 $us Tienda,2023-06-19,"39.800 $US tienda a estrenar, iglesia Cala Cal...",$us 39 800,Cochabamba - Venta - Local Comercial,/anuncio/inmuebles/39-800-us-tienda/2015299
2,Casa En Venta Superficie,2023-06-19,CASA En venta superficie 325 m2 ubicado en Av....,,Cochabamba - Venta - Casa / Chalet,/anuncio/inmuebles/casa-en-venta-superficie/20...
3,Casa 2 Pisos Garaje,2023-06-19,"CASA 2 pisos, garaje, 3 dormitorios, cocina, b...",,Cochabamba - Venta - Casa / Chalet,/anuncio/inmuebles/casa-2-pisos-garaje/2017770
4,Vendo Departamento América Casi,2023-06-19,"VENDO Departamento: América casi Libertador, 3...",,Cochabamba - Venta - Departamento,/anuncio/inmuebles/vendo-departamento-america-...
...,...,...,...,...,...,...
57,Departamento A Estrenar En,2023-06-17,"DEPARTAMENTO a estrenar, en alquiler, 82 m2, c...",,Cochabamba - Alquiler - Departamento,/anuncio/inmuebles/departamento-estrenar-en/20...
58,Departamento En Alquiler Zona,2023-06-17,"DEPARTAMENTO en alquiler, zona parque Kanata, ...",,Cochabamba - Alquiler - Departamento,/anuncio/inmuebles/departamento-en-alquiler-zo...
59,Alquiler Sobre Av Aniceto,2023-06-17,"ALQUILER Sobre Av. Aniceto Arce, departamento ...",,Cochabamba - Alquiler - Departamento,/anuncio/inmuebles/alquiler-sobre-av-aniceto/2...
60,2 Departamentos En Anticrético,2023-06-17,"2 DEPARTAMENTOS en anticrético, uno de 2 dormi...",,Cochabamba - Anticrético - Departamento,/anuncio/inmuebles/2-departamentos-en-anticret...


In [11]:
# Save dataframe as .csv file 
df.to_csv(f'./csv/los_tiempos_clasificados_{datetime.now().isoformat()[:10]}.csv', sep=';')